From Text to Word-adjacency networks

In [1]:
import networkx as nx

def get_network(text, disconnected_chars = ['.', ','], weighted=False):
    network = nx.DiGraph()
    words = text.split(" ")
    network.add_node(words[0])
    for (n, word) in enumerate(words[1:]):
        n+=1
        network.add_node(word)
        edge = (words[n-1], words[n])
        if edge in network.edges() and weighted:
            network.add_edge(edge[0], edge[1], weight=network.get_edge_data(*edge)['weight']+1)
        elif weighted: network.add_edge(*edge, weight=1)
        else: network.add_edge(*edge)
    
    network.remove_nodes_from(disconnected_chars)
    return network

Obtain text to analyse

In [2]:
from src.random_text import get_text

text = get_text(size=100)

Convert from Text-To-Network

In [6]:
network = get_network(text, weighted=False)
print(network.edges())

[('scarabaeus', 'coredeemer'), ('coredeemer', 'ask'), ('ask', 'prudelike'), ('prudelike', 'hemocrystallin'), ('hemocrystallin', 'theligonaceous'), ('theligonaceous', 'subhysteria'), ('subhysteria', 'irremissive'), ('irremissive', 'decalogist'), ('decalogist', 'bewreath'), ('bewreath', 'underpole'), ('underpole', 'begum'), ('begum', 'chullpa'), ('chullpa', 'bostonian'), ('bostonian', 'franciscan'), ('franciscan', 'regelate'), ('regelate', 'enleven'), ('enleven', 'adiposis'), ('adiposis', 'arcadia'), ('arcadia', 'pyralidid'), ('pyralidid', 'chancroid'), ('chancroid', 'claro-obscuro'), ('claro-obscuro', 'icebreaker'), ('icebreaker', 'thelphusidae'), ('thelphusidae', 'mainferre'), ('mainferre', 'smuisty'), ('smuisty', 'attribute'), ('attribute', 'ophthalmitic'), ('ophthalmitic', 'smiling'), ('smiling', 'testificatory'), ('testificatory', 'verbasco'), ('verbasco', 'suyog'), ('suyog', 'thort'), ('thort', 'obambulatory'), ('obambulatory', 'celiocolpotomy'), ('celiocolpotomy', 'yolkless'), ('y